# Load tapes and lob data

do this for each day otherwise memory (RAM) exceeds most computers

In [2]:
# code
from fast_tools import get_data

data = get_data(0,1)

Time taken to reach each day: 0.17849993705749512


(array([[ 0,  0,  0, ...,  0,  0,  0],
        [-6,  0,  0, ...,  0,  0,  0],
        [-6,  0,  0, ...,  0,  0,  1],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]], dtype=int8),
 array([0.0000000e+00, 2.7900000e-01, 1.3330000e+00, ..., 3.0599635e+04,
        3.0599759e+04, 3.0599945e+04]),
 array([[1.1067000e+01, 2.6900000e+02, 1.0000000e+00],
        [1.1222000e+01, 2.6700000e+02, 2.0000000e+00],
        [1.2338000e+01, 2.7000000e+02, 2.0000000e+00],
        ...,
        [3.0599108e+04, 2.9200000e+02, 1.0000000e+00],
        [3.0599728e+04, 2.9000000e+02, 1.0000000e+00],
        [3.0599728e+04, 2.8800000e+02, 1.0000000e+00]]))

# Clean Data
Remove outliers from lob and create an additional columns noting this

FFill tapes data to get the most up to date tapes price

In [ ]:
# code

# Extract features from LOB and Tapes

Use 1min aggregation

In [ ]:
# code

# Save features

for use in models